In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Reading in File . . .
file_path = "train.json"
json_data = []

df = pd.read_json(file_path, lines=True)
print(df.head())

C:\Users\Matt\AppData\Local\Temp\ipykernel_41092\892108931.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json(file_path, lines=True)
C:\Users\Matt\AppData\Local\Temp\ipykernel_41092\892108931.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json(file_path, lines=True)
C:\Users\Matt\AppData\Local\Temp\ipykernel_41092\892108931.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, str

       0.0      36.6      36.7      38.0      38.2  38.300000000000004  \
0   3.0074   81.2963   83.7169  119.6888  115.8881            413.4768   
1  13.6523  361.5547  119.7225  170.9554   84.0531            273.8909   
2   7.1155   80.2869  169.7490  149.4092  132.8215            198.8544   
3   2.0998  235.5117  161.2402  241.3578  243.2237             89.7297   
4  16.2152  384.2739  174.5714   84.3117  137.9726            261.8213   

       39.0      39.1      39.2       39.6  ...    466.0    523.0    523.1  \
0  136.8640  817.5328   86.4743   477.6116  ...  15.8853   1.5231   4.9432   
1  175.9083  665.0791  588.2624    84.1962  ...   6.9459  26.9344   9.0805   
2  150.5273  112.4005  335.4482   675.7621  ...   9.3406  18.8055   7.0021   
3  640.7394  121.9373  396.3036  1138.3905  ...  26.2306   8.1640  26.6106   
4  365.8102   96.3555  219.9515   211.2868  ...  39.9084   4.7010  50.1098   

     523.4     524.4    524.5     554.2     554.4    1046.0       target  
0   9.2590 

In [3]:
# Combining targets that are tonally equivalent . . .   
    
def combine_equal(df):
    tonal_equ = {
                "A#": "Bb",
                "B#": "C",
                "C#": "Db",
                "D#": "Eb",
                "E#": "F",
                "F#": "Gb",
                "G#": "Ab"
            }
    
    df[['note', 'modality']] = pd.DataFrame(df.target.tolist(), index=df.index)
    df["note"] = df['note'].replace(tonal_equ)
    df["target"] = df[["note", "modality"]].astype(str).apply(' '.join, axis=1)
    df.drop(["note", "modality"], axis=1, inplace=True)

combine_equal(df)

In [4]:
# Splitting into train test validation sets
X = df.drop('target', axis=1).values
label_encoder = LabelEncoder()
df['target_encoded'] = label_encoder.fit_transform(df['target'])
y = df['target_encoded'].values
x_train, x_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

# Testing function for the data split

print("Training x:", x_train.shape,"y:", y_train.shape)
print("Testing x:", x_test.shape,"y:", y_test.shape)
x_train

Training x: (7877, 1000) y: (7877,)
Testing x: (1970, 1000) y: (1970,)


array([[  0.3821,   0.497 ,   0.7169, ...,   4.9492,   0.896 ,   3.2916],
       [ 16.4461,  55.8946,  54.2504, ..., 163.5266, 155.7604,  10.2477],
       [  5.2831,   7.8013,   9.3183, ..., 104.2608, 134.1521,  32.3494],
       ...,
       [  5.6634,  25.6487,  38.779 , ...,  50.1856,  43.7502, 206.6477],
       [ 31.726 ,   7.6548,  39.6652, ..., 162.1529, 186.4395,  19.2206],
       [  0.3202, 169.3784,  94.1028, ...,  57.3187,  35.3127,  13.3427]])

In [5]:
# Testing decision tree
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)

score = clf.score(x_test, y_test)
score

0.23147208121827412

In [6]:
# Random Forest
cls = RandomForestClassifier()
cls.fit(x_train, y_train)

score = cls.score(x_test, y_test)
score

0.5172588832487309